# Comment Toxicity Challenge

In [2]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from bert_sklearn import BertClassifier

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


### Reading in all datasets
Here, we read in our datasets. We're only going to use the training set given by Kaggle, and we'll eventually end up splitting up the data with a 5% testing and 95% training split. 

In [3]:
train_set = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_test_labs = pd.read_csv("test_labels.csv")

### Cleaning up the data
In this step, we clean up the data. We use Python's regex library to do that (it takes in a regular expression and transforms it to the programmer's specifications.) For now, we only really want letters in our dataset - anything else is going to be replaced by a space. 

In [4]:
regex = re.compile('[^a-zA-Z]')
fixed = []
size = len(train_set.index)
size_that_my_computer_can_handle = 1000
starting_point = 5000
for i in range(starting_point, starting_point+size_that_my_computer_can_handle):
    temp = regex.sub(' ', train_set['comment_text'][i])
    temp = temp.lower()
    fixed.append(temp)

In [21]:
vec = TfidfVectorizer()
X = TfidfVectorizer().fit_transform(fixed).toarray()
print(X.shape)
y_tox = train_set.iloc[starting_point:size_that_my_computer_can_handle+starting_point,2].values
y_sev_tox = train_set.iloc[starting_point:size_that_my_computer_can_handle+starting_point,3].values
y_obs = train_set.iloc[starting_point:size_that_my_computer_can_handle+starting_point,4].values
y_threat = train_set.iloc[starting_point:size_that_my_computer_can_handle+starting_point,5].values
y_insult = train_set.iloc[starting_point:size_that_my_computer_can_handle+starting_point,6].values
y_hate = train_set.iloc[starting_point:size_that_my_computer_can_handle+starting_point,7].values

(1000, 8225)


### Gaussian Naive Bayes
Here, we assign, then run (in the next step) the Gaussian Naive Bayes model on our data set.

In [22]:
tests = {'y_tox' : y_tox, 
         'y_sev_tox' : y_sev_tox, 
         'y_obs' : y_obs, 
         'y_threat' : y_threat, 
         'y_insult' : y_insult, 
         'y_hate' : y_hate}

gnb_models = {'y_tox' : GaussianNB(), 
          'y_sev_tox' : GaussianNB(),
          'y_obs' : GaussianNB(),
          'y_threat' : GaussianNB(),
          'y_insult' : GaussianNB(),
          'y_hate' : GaussianNB()}

preds = {}

test_names = ['y_tox', 'y_sev_tox', 'y_obs', 'y_threat', 'y_insult', 'y_hate']

In [23]:
for i in test_names:
    print(X.shape)
    X_train, X_test, y_train, y_test = train_test_split(X, tests[i])
    print(X_train.shape)
    gnb_models[i].fit(X_train, y_train)
    print(X_test.shape)
    preds[i] = gnb_models[i].predict(X_test)
    print(i)
    print(confusion_matrix(y_test, preds[i]))
    print(accuracy_score(y_test, preds[i]))

(1000, 8225)
(750, 8225)
(250, 8225)
y_tox
[[201  27]
 [ 21   1]]
0.808
(1000, 8225)
(750, 8225)
(250, 8225)
y_sev_tox
[[241   6]
 [  2   1]]
0.968
(1000, 8225)
(750, 8225)
(250, 8225)
y_obs
[[219  13]
 [ 17   1]]
0.88
(1000, 8225)
(750, 8225)
(250, 8225)
y_threat
[[246   4]
 [  0   0]]
0.984
(1000, 8225)
(750, 8225)
(250, 8225)
y_insult
[[226  12]
 [ 11   1]]
0.908
(1000, 8225)
(750, 8225)
(250, 8225)
y_hate
[[242   6]
 [  2   0]]
0.968


In [26]:
regex = re.compile('[^a-zA-Z]')
fixed = []
size = len(train_set.index)
size_that_my_computer_can_handle = 1000
starting_point = 5000
for i in range(starting_point, starting_point+size_that_my_computer_can_handle):
    temp = regex.sub(' ', df_test['comment_text'][i])
    temp = temp.lower()
    fixed.append(temp)

print(X.shape)
#kaggle test array    
X_kaggle = vec.fit_transform(fixed).toarray()
print(X_kaggle.shape)
kaggle_preds = {}

(1000, 8225)
(1000, 8225)


In [25]:
for i in test_names:
    print(X_kaggle.shape)
    kaggle_preds[i] = gnb_models[i].predict(X_kaggle)
     
#list out in sample sub
sample_sub = pd.DataFrame(df_sub.iloc[130000:,0])
sample_sub = sample_sub.reset_index(drop=True)
sample_placement = pd.DataFrame(kaggle_preds)
sample_placement = sample_placement.reindex_axis(['y_tox','y_sev_tox', 'y_obs','y_threat', 'y_insult', 'y_hate'], axis=1)
sample_sub = sample_sub.join(sample_placement)
sample_sub = sample_sub.rename(columns={'y_tox': 'toxic', 'y_sev_tox': 'sever_toxic',
                                        'y_obs': 'obscene', 'y_threat': 'threat', 
                                        'y)insult': 'insult', 'y_hate': 'indentity_hate'})

(1000, 8225)
(1000, 8225)
(1000, 8225)
(1000, 8225)
(1000, 8225)
(1000, 8225)


NameError: name 'df_sub' is not defined

### Changing the vectorizer and re-running the model
What differences can we note? 

In [12]:
X = HashingVectorizer().fit_transform(fixed).toarray()
print(X)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
for i in test_names:
    X_train, X_test, y_train, y_test = train_test_split(X, tests[i])
    gnb_models[i].fit(X_train, y_train)
    preds[i] = gnb_models[i].predict(X_test)
    print(i)
    print(confusion_matrix(y_test, preds[i]))
    print(accuracy_score(y_test, preds[i]))

Let's change the vectorizer back. 

In [ ]:
X = CountVectorizer().fit_transform(fixed).toarray()

### SVM Model
Here, we assign, then run (in the next step) the Support Vector Machine model on our data set.

In [ ]:
svm_models = {'y_tox' : SVC(gamma = 'scale'), 
          'y_sev_tox' : SVC(gamma = 'scale'),
          'y_obs' : SVC(gamma = 'scale'),
          'y_threat' : SVC(gamma = 'scale'),
          'y_insult' : SVC(gamma = 'scale'),
          'y_hate' : SVC(gamma = 'scale')}

In [66]:
for i in test_names:
    X_train, X_test, y_train, y_test = train_test_split(X, tests[i])
    svm_models[i].fit(X_train, y_train)
    preds[i] = svm_models[i].predict(X_test)
    print(i)
    print(confusion_matrix(y_test, preds[i]))
    print(accuracy_score(y_test, preds[i]))

y_tox
[[173   0]
 [ 26   1]]
0.87
y_sev_tox
[[197   0]
 [  3   0]]
0.985
y_obs
[[190   0]
 [ 10   0]]
0.95
y_threat
[[199   0]
 [  1   0]]
0.995
y_insult
[[184   0]
 [ 16   0]]
0.92
y_hate
[[199   0]
 [  1   0]]
0.995


### K Nearest Neighbors Model
Here, we assign, then run (in the next step) the K Nearest Neighbors model on our data set.

In [ ]:
k_nearest_models = {'y_tox' : KNeighborsClassifier(), 
          'y_sev_tox' : KNeighborsClassifier(),
          'y_obs' : KNeighborsClassifier(),
          'y_threat' : KNeighborsClassifier(),
          'y_insult' : KNeighborsClassifier(),
          'y_hate' : KNeighborsClassifier()}

In [15]:
for i in test_names:
    X_train, X_test, y_train, y_test = train_test_split(X, tests[i])
    k_nearest_models[i].fit(X_train, y_train)
    preds[i] = k_nearest_models[i].predict(X_test)
    print(i)
    print(confusion_matrix(y_test, preds[i]))
    print(accuracy_score(y_test, preds[i]))

y_tox
[[438  12]
 [ 41   9]]
0.894
y_sev_tox
[[496   0]
 [  4   0]]
0.992
y_obs
[[471   4]
 [ 21   4]]
0.95
y_threat
[[499   0]
 [  1   0]]
0.998
y_insult
[[467   9]
 [ 22   2]]
0.938
y_hate
[[496   0]
 [  4   0]]
0.992


### Random Forest Model
Here, we assign, then run (in the next step) the Random Forest model on our data set.

In [35]:
random_forest_models = {'y_tox' : RandomForestClassifier(), 
          'y_sev_tox' : RandomForestClassifier(),
          'y_obs' : RandomForestClassifier(),
          'y_threat' : RandomForestClassifier(),
          'y_insult' : RandomForestClassifier(),
          'y_hate' : RandomForestClassifier()}

In [68]:
for i in test_names:
    X_train, X_test, y_train, y_test = train_test_split(X, tests[i])
    random_forest_models[i].fit(X_train, y_train)
    preds[i] = random_forest_models[i].predict(X_test)
    print(i)
    print(confusion_matrix(y_test, preds[i]))
    print(accuracy_score(y_test, preds[i]))

y_tox
[[85  1]
 [10  4]]
0.89
y_sev_tox
[[99  1]
 [ 0  0]]
0.99
y_obs
[[95  0]
 [ 4  1]]
0.96
y_threat
[[99  0]
 [ 1  0]]
0.99
y_insult
[[94  0]
 [ 5  1]]
0.95
y_hate
[[100]]
1.0


### Decision Tree Model
Here, we assign, then run (in the next step) the Decision Tree model on our data set.

In [16]:
decision_tree_models = {'y_tox' : DecisionTreeClassifier(), 
          'y_sev_tox' : DecisionTreeClassifier(),
          'y_obs' : DecisionTreeClassifier(),
          'y_threat' : DecisionTreeClassifier(),
          'y_insult' : DecisionTreeClassifier(),
          'y_hate' : DecisionTreeClassifier()}

In [70]:
for i in test_names:
    X_train, X_test, y_train, y_test = train_test_split(X, tests[i])
    decision_tree_models[i].fit(X_train, y_train)
    preds[i] = decision_tree_models[i].predict(X_test)
    print(i)
    print(confusion_matrix(y_test, preds[i]))
    print(accuracy_score(y_test, preds[i]))

y_tox
[[453   5]
 [ 28  14]]
0.934
y_sev_tox
[[494   1]
 [  5   0]]
0.988
y_obs
[[472   6]
 [  8  14]]
0.972
y_threat
[[498   0]
 [  2   0]]
0.996
y_insult
[[473   8]
 [  9  10]]
0.966
y_hate
[[493   3]
 [  4   0]]
0.986


### Logistic Regression
Here, we assign, then run (in the next step) the Logistic Regression model on our data set.

In [23]:
logistic_regression_models = {'y_tox' : LogisticRegression(solver = 'lbfgs', max_iter = 200), 
          'y_sev_tox' : LogisticRegression(solver = 'lbfgs', max_iter = 200),
          'y_obs' : LogisticRegression(solver = 'lbfgs', max_iter = 200),
          'y_threat' : LogisticRegression(solver = 'lbfgs', max_iter = 200),
          'y_insult' : LogisticRegression(solver = 'lbfgs', max_iter = 200),
          'y_hate' : LogisticRegression(solver = 'lbfgs', max_iter = 200)}

In [24]:
for i in test_names:
    X_train, X_test, y_train, y_test = train_test_split(X, tests[i])
    logistic_regression_models[i].fit(X_train, y_train)
    preds[i] = logistic_regression_models[i].predict(X_test)
    print(i)
    print(confusion_matrix(y_test, preds[i]))
    print(accuracy_score(y_test, preds[i]))

y_tox
[[447   6]
 [ 31  16]]
0.926
y_sev_tox
[[497   0]
 [  2   1]]
0.996
y_obs
[[475   2]
 [ 17   6]]
0.962
y_threat
[[499   1]
 [  0   0]]
0.998
y_insult
[[473   2]
 [ 18   7]]
0.96
y_hate
[[496   1]
 [  3   0]]
0.992


In [8]:
bert_models = {'y_tox' : BertClassifier(), 
          'y_sev_tox' : BertClassifier(),
          'y_obs' : BertClassifier(),
          'y_threat' : BertClassifier(),
          'y_insult' : BertClassifier(),
          'y_hate' : BertClassifier()}

Building sklearn classifier...
Building sklearn classifier...
Building sklearn classifier...
Building sklearn classifier...
Building sklearn classifier...
Building sklearn classifier...


In [ ]:
for i in test_names:
    X_train, X_test, y_train, y_test = train_test_split(X, tests[i])
    bert_models[i].fit(X_train, y_train)
    preds[i] = bert_models[i].predict(X_test)
    print(i)
    print(confusion_matrix(y_test, preds[i]))
    print(accuracy_score(y_test, preds[i]))

100%|██████████| 231508/231508 [00:00<00:00, 3974324.58B/s]


Loading bert-base-uncased model...


100%|██████████| 407873900/407873900 [00:30<00:00, 13361205.47B/s]


Defaulting to linear classifier/regressor
train data size: 1350, validation data size: 150


Training: 100%|██████████| 43/43 [4:28:35<00:00, 73.14s/it, loss=0.35]      
                                                           

Epoch 1, Train loss : 0.3504, Val loss: 0.3883, Val accy = 87.33%


Training:   2%|▏         | 1/43 [00:36<25:21, 36.22s/it, loss=0.25]